In [1]:
#SGN
#Importing the required libraries
import tweepy
from tweepy.auth import OAuthHandler
import json
import datetime
import pandas as pd


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, pos_tag_sents


import re
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
#Defining the Variables
APP_KEY = 'lXMh6z51CYTalPJo4Faf8yXW3'
APP_SECRET = 'fM7AEgtJt7tAaALQWJFWjgIvcD0aJpQ8eigUg609LM9QDcLEHF'
OATH_TOKEN = '152140286-NypzIPtzPyDOvE5vWzjfI2TkjOB9Q4WJFFXGRWB7'
OATH_TOKEN_SECRET = 'yBry40PjtELUxbCBLpKmHx983BecEtQMalwlnETkOOeDs'

In [5]:
Keywords = ["#Global Views","#Tele Groceries","#The Bakery Companies",
            "#BentonDread","#FreshDirect",
            "#Gillette","#Pepsi","#Pedigree",
           "#Phillips","#Gift Of Love","#Faber-Castell"]

In [ ]:
#Keywords = ["Tele Groceries","#Global Views"]

In [6]:
auth = OAuthHandler(APP_KEY, APP_SECRET)
auth.set_access_token(OATH_TOKEN, OATH_TOKEN_SECRET)

In [8]:
api = tweepy.API(auth)

In [9]:
targetDate_prev = datetime.datetime.today()-datetime.timedelta(days=70)
targetDate_prev = targetDate_prev.strftime('%Y-%m-%d')

In [10]:
targetDate_prev

'2019-11-21'

In [11]:
targetDate_today = datetime.datetime.today().strftime('%Y-%m-%d')

In [12]:
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date','Category'])

In [13]:
df_all = pd.DataFrame(columns=['Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date','Category'])

In [14]:
def stream(data):
        global df_all
        for word in data:
            i = 0
            for tweet in tweepy.Cursor(api.search, q=word, count=10, lang='en',since = targetDate_prev , unitl=targetDate_today).items():
                print(i, end='\r')
                df.loc[i, 'Tweets'] = tweet.text
                df.loc[i, 'User'] = tweet.user.name
                df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
                df.loc[i, 'user_followers'] = tweet.user.followers_count
                df.loc[i, 'User_location'] = tweet.user.location
                df.loc[i, 'User_verified'] = tweet.user.verified
                df.loc[i, 'fav_count'] = tweet.favorite_count
                df.loc[i, 'rt_count'] = tweet.retweet_count
                df.loc[i, 'tweet_date'] = tweet.created_at
                df.loc[i, 'Category'] = word
                i+=1
                df_all = df_all.append(df)
                if i == 100:
                    break
                else:
                    pass
                

In [ ]:
def stream(data):
    global df_all
    df_all = pd.DataFrame()
    for word in data:
        i = 0
        for tweet in tweepy.Cursor(api.search, q=word, count=100, lang='en',since = '2019-11-18' , unitl=targetDate_today).items():
            print(i, end='\r')
            if i <= 10:
                Tweet_dict = {'Tweets': [tweet.text],
                                'User' : [tweet.user.name],
                                'User_statuses_count' : [tweet.user.statuses_count],
                                'user_followers' : [tweet.user.followers_count],
                                'User_location' : [tweet.user.location],
                                'User_verified': [tweet.user.verified],
                                'fav_count':[tweet.favorite_count],
                                'rt_count': [tweet.retweet_count],
                                'tweet_date':[tweet.created_at],
                                'Category': [word]}
                df = pd.DataFrame(Tweet_dict)
                df_all = pd.concat([df_all,df],sort=False)
                i+=1 
            else:
                break
                

In [15]:
stream(data = Keywords)

In [16]:
df_all.shape

(6861359, 10)

In [18]:
df_copy=df_all

In [19]:
df_copy.head(4)

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,Category
0,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views
0,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views
1,RT @NtechYel: Fascinating #statistics of my #c...,Wesley Pennock,333,61,"San Antonio, TX",False,0,9,2020-01-29 21:16:00,#Global Views
0,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views


In [20]:
df_all.Category.value_counts()

#Sony             6249430
#Pepsi             459619
#Gift Of Love       84100
#Faber-Castell      57970
#Gillette            7921
#Pedigree            1681
#Global Views         589
#FreshDirect           49
Name: Category, dtype: int64

In [22]:
Product_Descr = {'Category': ['#Global Views','#Tele Groceries','#The Bakery Companies', '#BentonDread', '#FreshDirect', 
                              '#Gillette','#Pepsi','#Pedigree','#Sony','#Gifts of Love','#Faber-Castell'],
                 'Product_Category': ['Home_improvement','Grocery','Bakery','Ready_to_eat','Meat','Personal_Grooming','Drinks','Pets','Electronics','Gifts',
                                     'Books_Stationary']}

Product_Data = pd.DataFrame(Product_Descr )

In [23]:
Product_Data

,Category,Product_Category
0,#Global Views,Home_improvement
1,#Tele Groceries,Grocery
2,#The Bakery Companies,Bakery
3,#BentonDread,Ready_to_eat
4,#FreshDirect,Meat
5,#Gillette,Personal_Grooming
6,#Pepsi,Drinks
7,#Pedigree,Pets
8,#Sony,Electronics
9,#Gifts of Love,Gifts


In [24]:
df_copy.shape

(6861359, 10)

In [25]:
#Merging with the Item_Master Data

df_all_new = pd.merge(df_all,Product_Data,how='left',on='Category')

In [27]:
df_all_new.head(2)

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,Category,Product_Category
0,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views,Home_improvement
1,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views,Home_improvement


In [28]:
df=df_all_new

In [29]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [30]:
df['Tweets'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [ ]:
# def clean_up(tweet_df):
       
#     ###---------->Inital Text Pre-Processing
#     tweet_df['clean_tweet'] = tweet_df['Tweets'].str.replace('[^a-zA-Z\s\$\%\.]','')
#     tweet_df['clean_tweet'] = tweet_df['clean_tweet'].apply(lambda x: " ".join(x.lower() for x in x.split())) 
#     operators = set(('and', 'or', 'not','but'))
#     stop = set(stopwords.words('english')) - operators
#     tweet_df['clean_tweet']  =  tweet_df['clean_tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    
#     ###----------> Lemmatization 
#     lem = WordNetLemmatizer()
#     tweet_df['clean_tweet']  = tweet_df.apply(lambda row: lem.lemmatize(row['Tweets']),axis = 1)
    
#      ###----------> Tokenization and Part of Speech Tagging to all words in the reviews 
#     tweet_df['clean_tweet'] = pos_tag_sents(tweet_df['clean_tweet'].apply(word_tokenize).tolist())
    
#     return tweet_df

In [ ]:
df = clean_up(df)

In [25]:
df.head(5)

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,Category
0,RT A 160 set of 120 Faber Castell colored pencils,John HOTUJEC ™ 🌈 🍒,115840,688,"Kansas City, MO",False,0,2,2020-01-30 09:04:05,#Faber-Castell
1,A 160 set of 120 Faber Castell colored pencils,Brady Barnhart,6002,15849,"Seattle, WA",False,27,2,2020-01-30 09:01:41,#Faber-Castell
2,That one is a Faber Castell Polychromos,"Sara Rhys, illustrator",4442,1948,"Nomad, currently in Somerset",False,1,0,2020-01-30 08:58:00,#Faber-Castell
3,Ini pake faber castell yang classic,shafira sayang hualian,36270,771,Indonesia,False,0,0,2020-01-30 08:07:20,#Faber-Castell
4,FOR SALE A series of Anemone Pavonina undertak...,Mike Bingham,3416,108,"Hull, East Yorkshire, England",False,0,0,2020-01-30 07:34:34,#Faber-Castell


In [29]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

In [31]:
def analyze_sentiment_polarity(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

In [35]:
def analyze_sentiment_subjectivity(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.subjectivity

In [ ]:
# df['Sentiment'] = df['Tweets'].apply(lambda x: analyze_sentiment(x))
# df['Sentiment_Polarity'] = df['Tweets'].apply(lambda x: analyze_sentiment_polarity(x))
# df['Sentiment_Subjectivity'] = df['Tweets'].apply(lambda x: analyze_sentiment_subjectivity(x))

In [33]:
df['Sentiment_Polarity'] = df['Tweets'].apply(lambda x: analyze_sentiment_polarity(x))

In [36]:
df['Sentiment_Subjectivity'] = df['Tweets'].apply(lambda x: analyze_sentiment_subjectivity(x))

In [ ]:
# nouns = []
# for i in range(df['clean_tweet'].size):
#     nouns.append([])
#     for j in range(len(df.loc[i, 'clean_tweet'])):
#         if (df.loc[i, 'clean_tweet'][j][1] == 'NN' or
#                 df.loc[i, 'clean_tweet'][j][1] == 'NNS' or
#                 df.loc[i, 'clean_tweet'][j][1] == 'NNP' or
#                 df.loc[i, 'clean_tweet'][j][1] == 'NNPS' or
#                 df.loc[i, 'clean_tweet'][j][1] == 'JJ' or
#                 df.loc[i, 'clean_tweet'][j][1] == 'VB'):
#                 nouns[i].append(df.loc[i, 'clean_tweet'][j])

                
# df['tweets_pos'] = nouns

In [34]:
df.head(5)

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,Category,Product_Category,Sentiment_Polarity
0,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views,Home_improvement,0.000000
1,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views,Home_improvement,0.000000
2,RT Fascinating statistics of my content global...,Wesley Pennock,333,61,"San Antonio, TX",False,0,9,2020-01-29 21:16:00,#Global Views,Home_improvement,0.140625
3,Once this hits 4000 views were dropping the vi...,Wattz | #TalkAboutIt OUT NOW!,89203,1757,Norwich 🔰,False,0,0,2020-01-30 13:20:12,#Global Views,Home_improvement,0.000000
4,RT Fascinating statistics of my content global...,Wesley Pennock,333,61,"San Antonio, TX",False,0,9,2020-01-29 21:16:00,#Global Views,Home_improvement,0.140625


In [ ]:
import datetime as dt

df['Week_Date'] = df['tweet_date'].dt.date

In [ ]:
# Table 2: For Time Series Visualization
df_weekly=df.assign(
    Re_Tweets_Weekly=df['rt_count'],
).groupby('Week_Date').agg(dict(Re_Tweets_Weekly=len)).reset_index()

df_weekly.sort_values("Week_Date", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

df_weekly.head(5)

In [ ]:
from datalab.context import Context
import datalab.storage as storage
import datalab.bigquery as bq
import pandas as pd
from pandas import DataFrame
import time
from pandas.io import gbq

In [ ]:
#Exporting Tables

df.to_gbq(destination_table="Tweets_Data.Tweets_data",project_id='fnmacc-esquires',if_exists='replace')#Main Tweets Table
df_new_1.to_gbq(destination_table="Tweets_Data.Tweets_Summary",project_id='fnmacc-esquires',if_exists='replace')# Summary Table
df_weekly.to_gbq(destination_table="Tweets_Data.Weekly_Tweets",project_id='fnmacc-esquires',if_exists='replace')# Weekly Table
df_pos.to_gbq(destination_table="Tweets_Data.Top_Pos_Tweets",project_id='fnmacc-esquires',if_exists='replace')#Positive Tweets
df_neg.to_gbq(destination_table="Tweets_Data.Top_Neg_Tweets",project_id='fnmacc-esquires',if_exists='replace')#Negative Tweets